In [14]:
import os
from dotenv import load_dotenv
import pandas as pd
from sqlalchemy import create_engine
from IPython.display import display
import bcrypt



In [2]:
load_dotenv()

DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_ENTRADA = os.getenv("DB_ENTRADA")
DB_SAI = os.getenv("DB_SAI")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")

ENTRADA = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_ENTRADA}"
SAIDA = f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_SAI}'

engine_entrada = create_engine(ENTRADA)
engine_saida = create_engine(SAIDA)


query = "SELECT * FROM Administrador"
dados_originais_df = pd.read_sql(query, engine_entrada)
df_anon_admin = dados_originais_df.copy()


In [3]:
def anonimizar_telefone(valor):
    valor_str = str(valor)
    valor_anonimizado = ''.join('*' if caractere.isdigit() else caractere for caractere in valor_str)
    return valor_anonimizado

df_anon_admin["telefone"] = df_anon_admin["telefone"].map(anonimizar_telefone)

print("==== Dados anonimizados ====")
display(df_anon_admin[["telefone"]])


==== Dados anonimizados ====


,telefone
0,***********
1,***********
2,***********
3,***********
4,***********
5,***********
6,***********
7,***********
8,***********
9,***********


In [4]:
def anonimizar_cpf(valor):
    valor_str = str(valor)
    valor_anonimizado = ''.join('*' if caractere.isdigit() else caractere for caractere in valor_str)
    return valor_anonimizado

df_anon_admin["cpf"] = df_anon_admin["cpf"].map(anonimizar_cpf)

print("==== Dados anonimizados ====")
display(df_anon_admin[["cpf"]])

==== Dados anonimizados ====


,cpf
0,***********
1,***********
2,***********
3,***********
4,***********
5,***********
6,***********
7,***********
8,***********
9,***********


In [15]:
def criptografar_senha(senha):
    
    salt = bcrypt.gensalt()
    hash_senha = bcrypt.hashpw(senha.encode('utf-8'), salt)
    return hash_senha.decode('utf-8')

def verificar_senha(senha_digitada, senha_criptografada):

    return bcrypt.checkpw(senha_digitada.encode('utf-8'), senha_criptografada.encode('utf-8'))

df_anon_admin["senha"] = df_anon_admin["senha"].apply(criptografar_senha)

print("==== Dados anonimizados ====")
display(df_anon_admin[["senha"]])




==== Dados anonimizados ====


,senha
0,$2b$12$tzc7HzvYucXLD52trrG6fezRK6ALoU3E.OihFFt...
1,$2b$12$2kIQ9XaPv4QguPcceUVaUuh1hboWp07mH3VB/.O...
2,$2b$12$PpaBDxuGHAV/Veqf.Ev3GegKLep5FNpNGk5PuCj...
3,$2b$12$EIq4pZt.BLxqRe/mGw92r.1GWIKKhzHGoAYyz2U...
4,$2b$12$9IU0.6eZFPjjZce9mlpg4eSZa30dW8Hg4jF0BRk...
5,$2b$12$q8J76iwajL8ql6s1icBBueW04vj5TJP.WlLRv6e...
6,$2b$12$2T4wYdoQS3y1JNt5GL1w4OusxgL8tlyv4JNlY3d...
7,$2b$12$4BZVwItfZKXipW5Gx7JueOtWI7GloqgmgsTDJuX...
8,$2b$12$46tb/YR12YgitHGFftXYU.ak2Ce2w6MNopYFfJ3...
9,$2b$12$v5vmp9hbN4JHCMtfDr3DGelYG7We0adPEIcOVGo...


In [ ]:

def anonimizar_email(email):
    email = str(email)
    parte_local, dominio = email.split("@")
  
    if len(parte_local) > 3:
        parte_local_pseudo = parte_local[:0] + "****"
    else:
        parte_local_pseudo = parte_local + "****"
    return parte_local_pseudo + "@" + dominio

df_anon_admin["email"] = df_anon_admin["email"].apply(anonimizar_email)

print("==== Dados anonimizados ====")
display(df_anon_admin[["email"]])

==== Dados anonimizados ====


,email
0,****@eitruck.com
1,****@eitruck.com
2,****@eitruck.com
3,****@eitruck.com
4,****@eitruck.com
5,****@eitruck.com
6,****@eitruck.com
7,****@eitruck.com
8,****@eitruck.com
9,****@eitruck.com


In [12]:
df_anon_admin["nome_completo"] = list(map(
    lambda i: f"Administrador {i+1}", 
    range(len(df_anon_admin))
))

#Imprime a anonimização 
print("==== Dados anonimizados ====")
display(df_anon_admin[['nome_completo']])



==== Dados anonimizados ====


,nome_completo
0,Administrador 1
1,Administrador 2
2,Administrador 3
3,Administrador 4
4,Administrador 5
5,Administrador 6
6,Administrador 7
7,Administrador 8
8,Administrador 9
9,Administrador 10


In [13]:
display(df_anon_admin)

,id,telefone,cpf,nome_completo,email,senha
0,2,***********,***********,Administrador 1,****@eitruck.com,$2b$12$vODcaiPuMzkkGLBBLUuS3O.bLk9AGuM3GNEkfvk...
1,3,***********,***********,Administrador 2,****@eitruck.com,$2b$12$zGQguByxmqDZ9.7xWvTLS.bmePNiYMg7SeYkOfc...
2,4,***********,***********,Administrador 3,****@eitruck.com,$2b$12$2cl5frV6wLxSsHs5r6T/fOOPg.uRNN0e424PJRF...
3,5,***********,***********,Administrador 4,****@eitruck.com,$2b$12$.SyQ1Mn8VXdFT4MXUAOEFefzShe6Sx/LHcfGLWp...
4,7,***********,***********,Administrador 5,****@eitruck.com,$2b$12$Ho8ISFJ9Wtixic.0cdzxn.xfwNr6UiCzSCp3J5G...
5,8,***********,***********,Administrador 6,****@eitruck.com,$2b$12$.VFNj3NDSDY.IhAVkDLdkOuLq3ymb0juRvynlij...
6,9,***********,***********,Administrador 7,****@eitruck.com,$2b$12$Y.DYSCylaXy0tBWdqcA8fOfYSiI2ggNw4WcKvGX...
7,10,***********,***********,Administrador 8,****@eitruck.com,$2b$12$BuLO5f4xujWRp6.KSI1U2uxDL6d0OukcSdu8uuK...
8,11,***********,***********,Administrador 9,****@eitruck.com,$2b$12$eqPvwSCieXqW95NBA5/k1.46OGj7OEdfyEjNyBP...
9,12,***********,***********,Administrador 10,****@eitruck.com,$2b$12$qqGs3IJ5FvljPFE3PiO4Z.va4Ray/5zJ4f7F5mH...


In [17]:
dados_originais_df.to_csv('original_administrador.csv')
df_anon_admin.to_csv('pseudo_administrador.csv')

In [16]:
df_anon_admin.to_sql(name='administrador', con=engine_saida, if_exists='replace', index=False)

12